# MODIS16A2数据重采样

尝试对MODIS16A2数据进行重采样。这里使用广泛被采用的最大值合成法。

## 最大值合成法

记$\{PET_i\}$为一个栅格中的降雨量时间序列，则在该时间范围内，这一栅格中的降雨量值可以用序列中的最大值$\mathrm{max}(\{PET_i\})$代替。

$$ PET=\mathrm{max}(\{PET_i\}) $$

扩展到二维形式，使用循环解决
```python
import numpy as np
PET_sequence = np.array([[[PET_tij for j in range(0,j)]for i in range(0,i)]for t in range(0,t)])

PET = []
for t in range(0,t):
    PET_ij = []
    for i in range(0,i):
        PET_i = []
        for j in range(0,j):
            PET_std = max([PET_sequence[i][j] for t in range(0,t)])
            PET_i.append(PET_std)
        PET_ij.append(PET_i)
    PET.append(PET_ij)

PET = np.array(PET)
```
使用了四重嵌套循环进行合成，效率相当的低。
能否进行一些优化？

可以使用`xarray`库自带的方法取最大值。`numpy`好像也有?

In [1]:
import numpy as np

In [9]:
a = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 5]])
b = np.array([[4, 5, 1], [2, 3, 6], [7, 8, 9]])

np.amax([a, b], axis=0)


array([[4, 5, 3],
       [4, 5, 6],
       [7, 8, 9]])